In [ ]:
!pip install weaviate-client==4.6.1 --force-reinstall -U -q

In [ ]:
import boto3
import weaviate
import weaviate.classes as wvc

# Initialize AWS clients
ecs_client = boto3.client('ecs')
ec2_client = boto3.client('ec2') 

# Get the task ARN
response = ecs_client.list_tasks(cluster='weaviate-cluster', serviceName='weaviate-service')
task_arn = response['taskArns'][0]

# Get the ENI ID
response = ecs_client.describe_tasks(cluster='weaviate-cluster', tasks=[task_arn])
eni_id = next(
    detail['value']
    for task in response['tasks']
    for attachment in task['attachments']
    for detail in attachment['details']
    if detail['name'] == 'networkInterfaceId'
)

# Get the Weaviate IP
response = ec2_client.describe_network_interfaces(NetworkInterfaceIds=[eni_id])
WEAVIATE_IP = response['NetworkInterfaces'][0]['Association']['PublicIp']

print(f"Weaviate IP: {WEAVIATE_IP}")
%store WEAVIATE_IP

# Connect to Weaviate
client = weaviate.connect_to_custom(
    http_host=weaviate_ip,
    http_port="8080",
    http_secure=False,
    grpc_host=weaviate_ip,
    grpc_port="50051",
    grpc_secure=False
)

collection = client.collections.create(
    name="TestCollection",
    properties=[
        wvc.config.Property(
            name="title",
            data_type=wvc.config.DataType.TEXT
        )
    ]
)
print("Created collection.")
collections = client.collections.list_all()
print(collections)
client.close()